# Create Submission From Model

Run inference on the test images to create results ready for submission. This will create a folder called `submission` in the current directory.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch, torchvision
torch.__version__

In [ ]:
from detectron2.evaluation.xview_evaluation import *

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import os
import numpy as np
import json
import shutil
import cv2
import random
import glob

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Set some parameters.
LOCALIZATION_SUBMISSION_FOLDER = "SUBMISSION_LOCALIZATION"
LOCALIZATION_MODEL_CONFIG = "./detectron2_repo/configs/xview/mask_rcnn_R_50_FPN_3x.yaml"
DAMAGE_SUBMISSION_FOLDER = "SUBMISSION_DAMAGE"
# DAMAGE_MODEL_CONFIG = ""

In [ ]:
# Removes and creates folders (clears them).
shutil.rmtree(LOCALIZATION_SUBMISSION_FOLDER, ignore_errors=True)
shutil.rmtree(DAMAGE_SUBMISSION_FOLDER, ignore_errors=True)
os.mkdir(LOCALIZATION_SUBMISSION_FOLDER)
os.mkdir(DAMAGE_SUBMISSION_FOLDER)

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

# Load localization model.
cfg_localization = get_cfg()
cfg_localization.merge_from_file(LOCALIZATION_MODEL_CONFIG)
# Load localization checkpoint.
cfg_localization.MODEL.WEIGHTS = os.path.join(cfg_localization.OUTPUT_DIR, "model_0054999.pth")
cfg_localization.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # set the testing threshold for this model
predictor_localization = DefaultPredictor(cfg_localization)

# Load damage model.
# cfg_damage = get_cfg()
# cfg_damage.merge_from_file(DAMAGE_MODEL_CONFIG)
# Load damage checkpoint.
# cfg_damage.MODEL.WEIGHTS = os.path.join(cfg_damage.OUTPUT_DIR, "model_0054999.pth")
# cfg_damage.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # set the testing threshold for this model
# predictor_damage = DefaultPredictor(cfg_damage)

In [ ]:
image_filenames = sorted(glob.glob("data/test/images/*"))
for filename in image_filenames:
    # Load the image.
    image = cv2.imread(filename)
    
    # Choose the model type based on pre or post localization
    if filename.find("pre") >= 0:
        # LOCALIZATION model.
        outputs = predictor_localization(image)
    
        # get to the right format
        instances = outputs["instances"].to('cpu')
        # use RLE to encode the masks, because they are too large and takes memory
        # since this evaluator stores outputs of the entire dataset
        # Our model may predict bool array, but cocoapi expects uint8
        rles = [
            mask_util.encode(np.array(mask[:, :, None], order="F", dtype="uint8"))[0]
            for mask in instances.pred_masks
        ]
        for rle in rles:
            # "counts" is an array encoded by mask_util as a byte-stream. Python3's
            # json writer which always produces strings cannot serialize a bytestream
            # unless you decode it. Thankfully, utf-8 works out (which is also what
            # the pycocotools/_mask.pyx does).
            rle["counts"] = rle["counts"].decode("utf-8")
        instances.pred_masks_rle = rles
        instances.remove("pred_masks")
        prediction = {}
        prediction["instances"] = instances_to_json(instances)
        height = 1024
        width = 1024
        
        pred_image = get_xview_localization_pred_image(height, width, prediction)
        
        name = os.path.basename(filename)
        new_filename = name.replace("pre", "localization")[:-4] + "_prediction.png"
        submission_filename = os.path.join(LOCALIZATION_SUBMISSION_FOLDER, new_filename)
        
        print(submission_filename)
        cv2.imwrite(submission_filename, pred_image)
        
    elif filename.find("post") >= 0:
        # DAMAGE model.
        # TODO: pretty much a repeat of localization but for damage
        pass

In [ ]:
i = cv2.imread("data/test/images/test_pre_00000.png")

In [ ]:
plt.imshow(i)

In [ ]:
plt.imshow(pred_image)